In [ ]:
#Import of Python available libraries
import sculib

mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
mpi.ip = '10.96.254.85'

In [ ]:
def mpi_t6102a1(scu):
    #Original 2
    #Feed Indexer tests [316-000000-043] 
    print('Test 6.10.2-a1')
    '''
    will need to run once for every indexer position where sensor is installed
    default is band5c
    '''
    #Revisions:
    #0.1 remove for loop as each band sequence has to be setup manually
    #0.2 now using sculib

    #Test setups

    #Common setup
    config_name = 'HN_INDEX_TEST'

    #band_at_start = status_finalValue("acu.general_management_and_controller.feed_indexer_pos")
    #bands = [ band_at_start, 'Band 1', 'Band 2']
    bands_seq1 = [ 'Band 5c', 'Band 1', 'Band 2']
    bands_seq2 = [ 'Band 1', 'Band 2', 'Band 5c']
    bands_seq3 = [ 'Band 2', 'Band 5c', 'Band 1']
    bands = bands_seq1


    #el_start = 15
    el_start = 20
    el_speed = 0.1
    az_start = -90
    az_speed = 3
    #elevations = [15, 25, 35, 45, 55, 65, 75, 85]
    elevations = [20, 25, 35, 45, 55, 65, 75, 85] #until lower limit allows 15 degrees

    loop1 = 5

    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200

    #debug logic
    #'''
    loop1 = 1
    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    debug=True
    #'''

    #b. Feed Indexer: Stability – gravity, increments, static (6.10.2-a1)
    '''
        1) Manually move the Dish to 15° Elevation, any Azimuth angle (suitable for the current site conditions) 
        and FI at SPF Band5c and de-activate all axes
        2) Manually start the LDS (Lateral Displacement Sensor / Kinesis) data recording
        3) Activate the Elevation axis
            ◦ wait for 15 seconds to complete the activation process (note the current SPF Band position(1))
        4) Command the FI to the SPF B1 position,
            ◦  wait for 40 seconds to complete the move
        5) Command the FI to the SPF B2 position, 
            ◦ wait for 40 seconds to complete the move
        6) Command the FI to the initial position(1),
            ◦ wait for 40 seconds to complete the move
        7) Start the ACU data recording on the SCU using logging configuration TBD,
            ◦ wait for 60 seconds to stabilise
        8) Command elevation upwards in steps of 10° (15/25/35/45/55/65/75/85) at a maximum rate of 1°/s, 
            ◦ wait for 80 seconds after each step command
        9) Once at 85° repeat step 8) downwards to 15°
        10) Repeat steps 8) and 9) for a total of 5 cycles
        11) Stop the ACU data recording on the SCU
        12) De-activate the Elevation axis
        13) Manually stop the LDS data recording
        14) Extract the data on the SCU @ 5Hz sampling
        15) Rename the file to TBD (eg. 6p10p2a1_i.csv)
        16) Repeat the sequence 1) to 15) with the FI at the SPF Band 1 starting position
        17) Repeat the sequence 1) to 15) with the FI at the SPF Band 2 starting position

    (Total duration ~ 1h45min per SPF Band => 5h30min)

    (1) The script to sample the current Elevation angle position at the start of the script and use it 
    as the “applicable SPF Band” since the test will be repeated for different SPF Band positions.
    '''
    print('Band sequence {}'.format(bands)) 

    #Step 1
    scu.activate_dmc()
    scu.wait_duration(wait10*2)

    #Move to starting az
    scu.abs_azimuth(az_start, az_speed)
    scu.wait_duration(wait5)

    #Move to starting el
    scu.abs_elevation(el_start, el_speed)
    scu.wait_duration(wait10*1)

    #Move to starting band
    scu.move_to_band(bands[0])
    scu.wait_duration(wait10*5)

    #Step 2a
    print('Start LDS recording here:')
    #input()

    #Step 2b
    if scu.logger_state() == 'RECORDING':
        print('WARNING, already recording - attempting to stop and start a fresh logger')
        scu.stop_logger()  
        scu.wait_duration(wait5)
    if scu.logger_state() == 'STOPPED':
        print('Starting logger with config: {}'.format(config_name))
        scu.start_logger(config_name)
        scu.wait_duration(wait5)

    #Step 3
    # activate elevation axis?

    #Step 4
    band=bands[1]
    scu.move_to_band(band)
    scu.wait_duration(wait10*6)

    #Step 5
    band=bands[2]
    scu.move_to_band(band)
    scu.wait_duration(wait10*6)

    #Step 6
    band = bands[0]
    scu.move_to_band(band)
    scu.wait_duration(wait10*6) 

    #Step 7
    #recording already started

    #Loop Step 8 and Step 9
    i=0
    while i < loop1:
        i+=1
        print('Loop 1 - Step 8 to 9: {:.0f} of {:.0f}'.format(i, loop1))
    #    8) Command elevation upwards in steps of 10° (15/25/35/45/55/65/75/85) at a maximum rate of 1°/s, 
    #        ◦ wait for 80 seconds after each step command
        #Step 8
        for el_angle in elevations[1:]:
            scu.abs_elevation(el_angle, el_speed)
            scu.wait_duration(wait100) #tbd need longer wait here if slewing at 0.1 deg.
            scu.wait_duration(wait10*6) 

        #Step 9
        for el_angle in list(reversed(elevations))[1:]:
            scu.abs_elevation(el_angle, el_speed)
            scu.wait_duration(wait100)
            scu.wait_duration(wait10*6) 

        #Step 10 - end loop

    #Step 11
    #stop recording
    scu.stop_logger()
    scu.wait_duration(wait5)
    print(scu.logger_state())

    #Step 12


    #Step 13
    print("Stop LDS data recording")

    #Step 14
    print("Extract the data on the SCU @ 5Hz sampling")

    #Step 15
    print("Rename the file to TBD (eg. 6p10p2a1_i.csv)")
    scu.save_session('6p10p2a1_i_' + bands[0], interval_ms = sampling)
    #Step 16
    # sequence 1) to 15) with the FI at the SPF Band 1 starting position
    #Step 17
    # the sequence 1) to 15) with the FI at the SPF Band 2 starting position


    scu.deactivate_dmc()
    scu.wait_duration(wait10*2)


In [ ]:
mpi_t6102a1(mpi)
